In [42]:
%load_ext autoreload
%autoreload 2
from sklearn import datasets
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from lark import Lark


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
def sklearn_to_df(sklearn_dataset):
    """ 
    Convert sklearn dataset to a dataframe, the class-label is renamed to "target"
    """
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df['target'] = pd.Series(sklearn_dataset.target)
    return df

In [44]:
target = "target"
dataset = datasets.load_iris()
dataset[target] = np.where(dataset[target]==2, 0, dataset[target])

In [45]:
# get df
data = sklearn_to_df(dataset)

# get X,y
X = data.drop(['target'], axis=1)
y = data['target']

X.columns = ['sepal length', 'sepal width', 'petal length', 'petal width']


# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1) # 70% training and 30% test

In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

clf_rf=RandomForestClassifier(n_estimators=100)
clf_mlp = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)

clf_rf.fit(X_train,y_train)
clf_mlp.fit(X_train,y_train)

y_pred=clf_mlp.predict(X_test)

In [47]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 1.0


In [48]:
print(clf_rf.predict([[3, 5, 4, 2]]))
print(clf_mlp.predict([[3, 5, 4, 2]]))

[1]
[0]


In [49]:
# create a random generator
import random
def random_generator(X):
    num_attributes = len(X.columns)
    x=[]
    for i in range(num_attributes):
        x.append(random.uniform(X[X.columns[i]].min(),X[X.columns[i]].max()))
    return x



In [50]:
for i in range(5):
    x = random_generator(X)
    print(x)
    print(clf_rf.predict([x]))
    print(clf_mlp.predict([x]))

[6.60855768568056, 3.293195797591048, 4.462011297037137, 1.6968951785104016]
[1]
[1]
[7.784575200990879, 2.856513895684748, 4.232243118740809, 2.097491302512731]
[0]
[1]
[7.236405261030885, 2.2155444264201014, 1.5414916198438577, 0.8998811117037129]
[0]
[0]
[7.812198266356409, 2.0206581966667323, 3.9308136101433417, 1.321655492769826]
[1]
[1]
[4.441497236857662, 2.767386901577291, 4.016799763634529, 1.0988640818628692]
[1]
[1]


In [51]:
from blackbox import BlackBox
bb = BlackBox(clf_mlp, clf_mlp.predict)

In [52]:
bb.classify_example([3, 5, 4, 2])

0

In [53]:
from sygus_if import SyGuS_IF
X = [
    [1,2,1,1],
    [-1.1,1,1,42]
]
y = [
    1,
    0
]

In [54]:
sgf = SyGuS_IF()
# sgf.fit(X,y)
sgf.fit(X_train,y_train)
print(sgf.synthesized_function)

(define-fun func ((sepal_length Real) (sepal_width Real) (petal_length Real) (petal_width Real)) Real (ite (> sepal_width 2) (ite (not (= petal_width (/ 1 2))) (ite (> petal_length 1) (ite (<= sepal_width 4) (ite (< sepal_width 4) (ite (>= petal_length (/ 4 3)) (ite (>= sepal_length 5) (ite (not (= petal_width 1)) (ite (>= sepal_width (/ 5 2)) (ite (> sepal_length 5) (ite (>= petal_length (/ 3 2)) (ite (> petal_length (/ 3 2)) (ite (>= petal_width (/ 1 3)) (ite (>= petal_width (/ 1 2)) (ite (>= petal_length 4) (ite (> petal_length 4) (ite (>= sepal_length 6) (ite (>= petal_width (/ 3 2)) (ite (not (= petal_width 2)) (ite (<= petal_length 6) (ite (< petal_length 6) (ite (> sepal_length 6) (ite (>= petal_length 5) (ite (<= petal_width 2) (ite (<= sepal_width 3) (ite (< sepal_width 3) (ite (<= sepal_width (/ 5 2)) 0 0) (ite (= petal_length 5) (- sepal_width 2) 0)) 0) 0) (ite (<= sepal_width 3) (ite (< sepal_width 3) (ite (<= sepal_width (/ 5 2)) (- petal_width (/ 1 2)) (ite (= petal_width

In [55]:
import time 

start_ = time.time()

y_pred_test = sgf.predict(X_test, y_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_test))
y_pred_train = sgf.predict(X_train, y_train)
print("Accuracy:",metrics.accuracy_score(y_train, y_pred_train))

print(time.time() - start_)

Accuracy: 0.8666666666666667
Accuracy: 1.0
2.8784189224243164


In [56]:
start_ = time.time()



y_pred_test = sgf.predict_z3(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_test))
y_pred_train = sgf.predict_z3(X_train)
print("Accuracy:",metrics.accuracy_score(y_train, y_pred_train))

print(time.time() - start_)

Accuracy: 0.8666666666666667
Accuracy: 1.0
1.9667532444000244


In [40]:
sgf._eval('(/ 39 49.0)')

0.7959183673469388